# A comparative review of patient matching approaches (Deduplication)
**This notebook compares different patient mactching approaches while demonstrating those concepts using python** </br>
**05 May 2020**

**Mayer Antoine**<br /> 
Public Health Informatics Fellow <br /> 
DGHT/Health Informatics Team <br /> 

## Introduction and Background

For the past years HIV program in PEPFAR supported countries have committed significant effort and resources to achieve the 95-95-95 target to end the AIDS by 2030. However, to successfully monitor their progress on the fast track strategy they need national patient-level longitudinal data repository of people living with HIV, tracking their HIV service and care from testing to death. To create such complete longitudinal patient record, countries need to link and deduplicate, on a regular basis, record from multiple data sources such as testing registry, art patient monitoring system, pharmacy-dispensing system, laboratory information system and case-based surveillance system.  

Patient matching strategies has been categorized over the years in 3 main categories: **deterministic, probabilistic and modern or machine learning**.
Choosing and developing the best approach to use in a given situation to match or deduplicate HIV patient records depends on the context, data quality, skills and resources available

Several algorithm and tools has been presented and implemented in multiple studies, also previous research has compared deterministic and probabilistic matching  but a few offer a concise comparison of the different approaches and provides practical guidance of selecting  an algorithm and operationalizing the model in a production environment either  for patient care or for HIV public health surveillance.  Our aim in this paper is to compare traditional and modern approach of patient matching techniques.

## Methods

We conducted an experiment comparing the different approaches by looking the effectiveness of some the prominent algorithm available in each approach. As shown in table 2, we selected 2 algorithms for each approach. Each algorithm was used to deduplicate a synthetic datasets and link two synthetic datasets. 

| Matching Approach| Algorithm |
| --- | --- | 
| Deterministic | Exact matching of patient demographics | 
|   | Composite Key OR pseudo identifier| 
| Probabilistic | Threshold-based Similarity Sum (SimSum) | 
|  |Threshold-based Weighted average|
|  | Expectation Maximization (EM) |
| Machine learning | Naive Bayes |
|  | Support Vector Machine (SVM) | 
|  |Logistic Regression |



The python record linkage toolkit (PRLT) was used during all steps of the experiment. It’s a library to link records in or between data sources 

## Patient matching process (deduplication)

![record linkage process](./docs/matching_process_dedup.PNG "record linkage")

## Experimental evaluation

In [6]:
#%load_ext autoreload
%reload_ext autoreload

In [7]:
%autoreload 2
import numpy as np
import pandas as pd

import patientlinkr as linkr
from metaphone import doublemetaphone

import recordlinkage as rl
from recordlinkage.preprocessing import phonetic,clean
from recordlinkage.datasets import load_febrl2,load_febrl1,load_febrl3,load_febrl4
from recordlinkage.datasets.febrl import _febrl_links
from recordlinkage.index import Block,SortedNeighbourhood
from recordlinkage.classifiers import KMeansClassifier , ECMClassifier ,NaiveBayesClassifier,SVMClassifier,LogisticRegressionClassifier

import warnings
warnings.filterwarnings("ignore")

#### Get version information

In [8]:
# Get Version information
print("Pandas version: {0}".format(pd.__version__),'\n')
print("Python Record Linkage version: {0}".format(rl._version.get_versions()['version']),'\n')
print("Numpy version: {0}".format(np.__version__),'\n')

Pandas version: 0.25.0 

Python Record Linkage version: 0.14 

Numpy version: 1.16.3 



In [9]:
# So all output comes through from Ipython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##  Importing dataset to deduplicate
Before we start with the deduplication process first we import the data in notebook. We assume the data has been cleaned and mapped to be imported in the format below :

| Variable| Description | Format |
| --- | --- | --- |
| rec_id | Internal ID in your data | |
| given_name | patient first name | |
| surname | patient last name | |
| date_of_birth | patient date of birth | YYYYMMDD  |
| sex | patient sex | |

In this notebook we use  synthetic dataset from The Python Record Linkage Toolkit (PRLT). The PRLT contains several open public synthetic datasets. The package is distributed with a  four synthetic datasets. For this project we will use The Freely Extensible Biomedical Record Linkage (Febrl) dataset 3 . 

In [10]:
import datetime

def run_import_data(file_name):
    # file to deduplicate
    IMPORT_FILE_TO_DEDUPLICATE = "./datasets/"+file_name
    print(IMPORT_FILE_TO_DEDUPLICATE)

    df_a = pd.read_csv(IMPORT_FILE_TO_DEDUPLICATE,
                        index_col="rec_id",
                        sep=",",
                        engine='c',
                        skipinitialspace=True,
                        encoding='utf-8',
                        parse_dates=["date_of_birth"])
    #df_a = df_a.drop(['culture','blocking_number'],axis =1)
    df_true_links = _febrl_links(df_a).to_frame(index=False)
    df_true_links.columns=['rec_id_1','rec_id_2']
    df_true_links.set_index(['rec_id_1','rec_id_2'],inplace=True)
    df_a = df_a[['given_name','surname','sex','date_of_birth']]
    df_a.sort_index().head()
    
    return df_a,df_true_links


#### Data-preprocessing

In [11]:

def run_preprocessing(df_a):
    df_a['given_name'] = clean(df_a['given_name'])
    df_a['surname'] = clean(df_a['surname'])
    #df_a['date_of_birth'] = clean(df_a['date_of_birth'])

    df_a['date_of_birth'] = pd.to_datetime(df_a['date_of_birth'],format='%Y%m%d', errors='coerce')
    df_a['YearB'] = df_a['date_of_birth'].dt.year.astype('Int64') # df_a['date_of_birth'].str[:4].astype(str)
    df_a['MonthB'] = df_a['date_of_birth'].dt.month.astype('Int64')  # df_a['date_of_birth'].str[5:7].astype(str)
    df_a['DayB'] = df_a['date_of_birth'].dt.day.astype('Int64')  # df_a['date_of_birth'].str[6:].astype(str)

    #df_a['metaphone_given_name'] = phonetic(df_a['given_name'], method='metaphone')
    #df_a['metaphone_surname'] = phonetic(df_a['surname'], method='metaphone')

    df_a['dbmetaphone_given_name'] =df_a['given_name'].apply(lambda x: doublemetaphone(str(x))[0] if(np.all(pd.notnull(x))) else x)
    df_a['dbmetaphone_surname'] = df_a['surname'].apply(lambda x: doublemetaphone(str(x))[0] if(np.all(pd.notnull(x))) else x)
    df_a['pseudo_unique_id'] = linkr._create_pseudo_id(df_a)
    
    return df_a


#### Deduplication function

In [19]:

def run_classification(df_a,df_true_links):
    
    df_a = df_a.copy()
    df_true_links = df_true_links.copy()

    blocks =[["dbmetaphone_given_name","dbmetaphone_surname"],
                  ["dbmetaphone_given_name", "date_of_birth"],
                  ["dbmetaphone_surname","sex"],
                  ["dbmetaphone_surname", "date_of_birth"],
                ['date_of_birth']]

    ## exact matching rules
    b = [['given_name','surname','date_of_birth',],
            ['given_name','surname']]

    ## pseudo_unique identifier
    bunique = [['pseudo_unique_id']]


    # #string comparison configuration
    comparison = [{ "vartype":"string", "field": "given_name","method":"jarowinkler", "threshold":0.85, "code":"given_name"},
                             { "vartype":"string", "field": "surname","method":"jarowinkler", "threshold":0.85, "code":"surname"},
                             { "vartype":"exact", "field": "date_of_birth", "code":"date_of_birth"},
                             { "vartype":"exact", "field": "sex","code":"sex"}

                     ]

    comparison_complex = [{ "vartype":"string", "field": "given_name","method":"jarowinkler", "code":"given_name"},
                             { "vartype":"string", "field": "surname","method":"jarowinkler", "code":"surname"},
                             { "vartype":"exact", "field": "date_of_birth", "code":"date_of_birth"},
                             { "vartype":"exact", "field": "sex","code":"sex"},
                     ]

    weigth_factor = {'given_name':2,
                     'surname':2, 
                     'date_of_birth':1,
                     'sex':1}


    approach = ["Deterministic","Probabilistic","Machine learning"]

    # list of classifiers to compare
    classifiers = [("Exact Matching", linkr.ExactMatchingClassifier()),
                   ("Pseudo Unique ID", linkr.ExactMatchingClassifier()),
                   ("ECM", ECMClassifier()),
                   ("SimSum",linkr.SimSumClassifier(threshold = 3)),
                   ("WeightedAverage",linkr.WeightedAverageClassifier(0.75,weigth_factor)),
                   ("NaivesBayes",NaiveBayesClassifier()),
                   ("SVM",SVMClassifier()),
                   ("LogisticRegression",LogisticRegressionClassifier()),
                  ]
    metrics = {}

    print('Number of records:',len(df_a))
    # iterate over classifiers
    for name, classifier in classifiers:

        # determnistic
        if(name == 'Exact Matching'):
            indexer = linkr.BlockUnion(block_on = b)
            pairs = indexer.index(df_a)
            match = classifier.fit_predict(pairs)
            nunique  = linkr.get_unique(df_a,match)
            matrix,precision,recall,fscore = linkr.metrics(df_true_links,match,pairs.to_frame(index=True))
            TP,FP,TN,FN = matrix[0,0],matrix[1,0],matrix[1,1],matrix[0,1]
            match_len = len(match)


        elif(name == 'Pseudo Unique ID'):
            indexer = linkr.BlockUnion(block_on = bunique) 
            pairs = indexer.index(df_a)
            match = classifier.fit_predict(pairs)
            nunique  = linkr.get_unique(df_a,match)
            matrix,precision,recall,fscore = linkr.metrics(df_true_links,match,pairs.to_frame(index=True))
            TP,FP,TN,FN = matrix[0,0],matrix[1,0],matrix[1,1],matrix[0,1]
            match_len = len(match)


        # probabilistic
        elif(name in ['ECM']):
            indexer = linkr.BlockUnion(block_on = blocks)
            pairs = indexer.index(df_a)  
            compare_cl = linkr.Comparator(compare_on = comparison)
            comparison_vectors = compare_cl.compute_compare(pairs,df_a)
            match = classifier.fit_predict(comparison_vectors)
            nunique  = linkr.get_unique(df_a,match)
            matrix,precision,recall,fscore = linkr.metrics(df_true_links,match,comparison_vectors)
            TP,FP,TN,FN = matrix[0,0],matrix[1,0],matrix[1,1],matrix[0,1]
            match_len = len(match)

        elif(name in ['SimSum','WeightedAverage']):
            indexer = linkr.BlockUnion(block_on = blocks)
            pairs = indexer.index(df_a)  
            # note complex comparison
            compare_cl = linkr.Comparator(compare_on = comparison_complex)
            comparison_vectors = compare_cl.compute_compare(pairs,df_a)
            match = classifier.fit_predict(comparison_vectors)
            nunique  = linkr.get_unique(df_a,match)
            matrix,precision,recall,fscore = linkr.metrics(df_true_links,match,comparison_vectors)
            TP,FP,TN,FN = matrix[0,0],matrix[1,0],matrix[1,1],matrix[0,1]
            match_len = len(match)

        # machine learning 
        elif(name in ['NaivesBayes','SVM','LogisticRegression']):
            indexer = linkr.BlockUnion(block_on = blocks)
            pairs = indexer.index(df_a)  
            compare_cl = linkr.Comparator(compare_on = comparison)
            comparison_vectors = compare_cl.compute_compare(pairs,df_a)      
            #note cross validation
            match = linkr.cross_val_predict(classifier,comparison_vectors,df_true_links,cv=10,method = 'predict')
            nunique  = linkr.get_unique(df_a,match)
            matrix,precision,recall,fscore = linkr.metrics(df_true_links,match,comparison_vectors)
            TP,FP,TN,FN = matrix[0,0],matrix[1,0],matrix[1,1],matrix[0,1]
            match_len = len(match)


        precision = round(precision*100,2)
        recall = round(recall*100,2)
        fscore = round(fscore*100,2)
        metrics[name] = [len(pairs),match_len,TP,FP,FN,precision,recall,fscore,nunique]
        print("Score for %s: %0.1f%% " % (name, fscore))
        
    return metrics
    



## Running the experiment

#### Dataset A
* **Dataset-A contains 5000 records (2500 originals and 25000 duplicates)**.
* **one duplicate maximum per records**
* **one modification maximum per field**
* **one modification maximum per record** 

In [21]:
df_a,df_true_links = run_import_data('dataset_a.csv')
df_a = run_preprocessing(df_a)
metrics = run_classification(df_a,df_true_links)
df_results = pd.DataFrame(metrics,index= ['pairs','matched','TP','FP','FN','presicion','recall','fscore','nunique'])
df_results_t = df_results.T.sort_values('fscore',ascending=False)
df_results_t.to_csv("dataset_a_results.csv")
df_results_t['fscore'].mean()
df_results_t

./datasets/dataset_a.csv
Number of records: 5000
Score for Exact Matching: 31.1% 
Score for Pseudo Unique ID: 80.4% 
Score for ECM: 95.9% 
Score for SimSum: 80.0% 
Score for WeightedAverage: 93.4% 
Score for NaivesBayes: 97.5% 
Score for SVM: 97.9% 
Score for LogisticRegression: 97.9% 


84.27499999999999

,pairs,matched,TP,FP,FN,presicion,recall,fscore,nunique
SVM,6665.0,2399.0,2399.0,0.0,101.0,100.00,95.96,97.94,2601.0
LogisticRegression,6665.0,2399.0,2399.0,0.0,101.0,100.00,95.96,97.94,2601.0
NaivesBayes,6665.0,2392.0,2385.0,7.0,115.0,99.71,95.40,97.51,2612.0
ECM,6665.0,2316.0,2309.0,7.0,191.0,99.70,92.36,95.89,2688.0
WeightedAverage,6665.0,2191.0,2190.0,1.0,310.0,99.95,87.60,93.37,2809.0
Pseudo Unique ID,1768.0,1768.0,1715.0,53.0,785.0,97.00,68.60,80.37,3255.0
SimSum,6665.0,1728.0,1692.0,36.0,808.0,97.92,67.68,80.04,3291.0
Exact Matching,461.0,461.0,461.0,0.0,2039.0,100.00,18.44,31.14,4539.0


#### Dataset B
* **Dataset-B contains 5000 records (4000 originals and 1000 duplicates)**.
* **5 duplicates maximum per records**
* **2 modifications maximum per field**
* **2 modifications maximum per record** 

In [14]:
df_a,df_true_links = run_import_data('dataset_b.csv')
df_a = run_preprocessing(df_a)
metrics = run_classification(df_a,df_true_links)
df_results = pd.DataFrame(metrics,index= ['pairs','matched','TP','FP','FN','presicion','recall','fscore','nunique'])
df_results_t = df_results.T.sort_values('fscore',ascending=False)
df_results_t.to_csv("dataset_b_results.csv")
df_results_t['fscore'].mean()
df_results_t

./datasets/dataset_b.csv
Score for Exact Matching: 14.8% 
Score for Pseudo Unique ID: 64.7% 
Score for ECM: 87.8% 
Score for SimSum: 67.1% 
Score for WeightedAverage: 87.8% 
Score for NaivesBayes: 88.5% 
Score for SVM: 92.5% 
Score for LogisticRegression: 92.5% 


74.4575

,pairs,matched,TP,FP,FN,presicion,recall,fscore,nunique
SVM,5278.0,862.0,861.0,1.0,139.0,99.88,86.1,92.48,4138.0
LogisticRegression,5278.0,862.0,861.0,1.0,139.0,99.88,86.1,92.48,4138.0
NaivesBayes,5278.0,803.0,798.0,5.0,202.0,99.38,79.8,88.52,4197.0
ECM,5278.0,792.0,787.0,5.0,213.0,99.37,78.7,87.83,4208.0
WeightedAverage,5278.0,795.0,788.0,7.0,212.0,99.12,78.8,87.80,4205.0
SimSum,5278.0,586.0,532.0,54.0,468.0,90.78,53.2,67.09,4429.0
Pseudo Unique ID,550.0,550.0,501.0,49.0,499.0,91.09,50.1,64.65,4461.0
Exact Matching,80.0,80.0,80.0,0.0,920.0,100.00,8.0,14.81,4920.0


#### Dataset C
* **Dataset-C contains 5000 records (3000 originals and 2000 duplicates)**.
* **5 duplicates maximum per records**
* **2 modifications maximum per field**
* **2 modifications maximum per record** 

In [15]:

df_a,df_true_links = run_import_data('dataset_c.csv')
df_a = run_preprocessing(df_a)
metrics = run_classification(df_a,df_true_links)
df_results = pd.DataFrame(metrics,index= ['pairs','matched','TP','FP','FN','presicion','recall','fscore','nunique'])
df_results_t = df_results.T.sort_values('fscore',ascending=False)
df_results_t.to_csv("dataset_c_results.csv")
df_results_t['fscore'].mean()
df_results_t

./datasets/dataset_c.csv
Score for Exact Matching: 16.3% 
Score for Pseudo Unique ID: 62.5% 
Score for ECM: 92.0% 
Score for SimSum: 67.7% 
Score for WeightedAverage: 87.9% 
Score for NaivesBayes: 92.0% 
Score for SVM: 92.0% 
Score for LogisticRegression: 92.0% 


75.3025

,pairs,matched,TP,FP,FN,presicion,recall,fscore,nunique
SVM,5965.0,1709.0,1707.0,2.0,293.0,99.88,85.35,92.05,3292.0
ECM,5965.0,1714.0,1709.0,5.0,291.0,99.71,85.45,92.03,3287.0
NaivesBayes,5965.0,1714.0,1709.0,5.0,291.0,99.71,85.45,92.03,3287.0
LogisticRegression,5965.0,1713.0,1708.0,5.0,292.0,99.71,85.40,92.00,3288.0
WeightedAverage,5965.0,1575.0,1571.0,4.0,429.0,99.75,78.55,87.89,3427.0
SimSum,5965.0,1068.0,1038.0,30.0,962.0,97.19,51.90,67.67,3939.0
Pseudo Unique ID,1015.0,1015.0,942.0,73.0,1058.0,92.81,47.10,62.49,4014.0
Exact Matching,177.0,177.0,177.0,0.0,1823.0,100.00,8.85,16.26,4823.0
